<a href="https://colab.research.google.com/github/sfnesbit/CSC466-Team13/blob/main/CSC_466_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 466 Team 13 Food Image Classification
Ty Farris | Marine Cossoul | Sean Nesbit

[Multiclass image classification article](https://towardsdatascience.com/end-to-end-pipeline-for-setting-up-multiclass-image-classification-for-data-scientists-2e051081d41c)

[Also](https://medium.com/swlh/convolutional-neural-networks-for-multiclass-image-classification-a-beginners-guide-to-6dbc09fabbd)

##  Mount drive
*You may need to visit CSC466 folder, and 'Add Shortcut to Drive' or 'Move To > My Drive"*

In [2]:
from google.colab import drive
drive.mount('/content/drive')
datadir = "drive/MyDrive/CSC466/csc466_data/"

Mounted at /content/drive


In [3]:
!ls drive/MyDrive/CSC466/

 csc466_data
'CSC 466 Project.ipynb'
'CSC 466 Team 13 Project Pitch.gdoc'
 food.zip
'KDD Project Proposal - Warmup Group 13.gdoc'


#  Create train and test datasets

In [8]:
import os
import cv2 as cv
from matplotlib import pyplot as plt

image_train_path = datadir+"images_train"
x_train = []
y_train = []

count = 0
for dir in os.listdir(image_train_path):
  print("directory: {0}".format(count))
  count = count + 1

  dir_path = image_train_path + "/" + dir
  for file in os.listdir(dir_path):
    #read in the image
    imagePath = image_train_path + "/" + dir + "/" + file
    image = cv.imread(imagePath)

    #resize the image to 128x128
    image = cv.resize(image, (128, 128)) 

    #add the image and corresponding label to x_train and y_train
    x_train.append(image)
    y_train.append(dir)
    
print()

#save results to a file
x_train = np.array(x_train, dtype="float") / 255.0
np.save(image_train_path + "/x_train.npy", x_train)

y_train = np.array(y_train)
np.save(image_train_path + "/y_train.npy", y_train)

directory: 0



In [ ]:
import os
import cv2 as cv
from matplotlib import pyplot as plt

image_test_path = datadir+"images_test"
x_test = []
y_test = []

count = 0
for dir in os.listdir(image_test_path):
  print("directory: {0}".format(count))
  count = count + 1
  
  dir_path = image_test_path + "/" + dir
  for file in os.listdir(dir_path):
    #read in the image
    imagePath = image_test_path + "/" + dir + "/" + file
    image = cv.imread(imagePath)

    #resize the image to 128x128
    image = cv.resize(image, (128, 128)) 

    #add the image and corresponding label to x_test and y_test
    x_test.append(image)
    y_test.append(dir)

#save results to a file
x_test = np.array(x_test, dtype="float") / 255.0
np.save(image_test_path + "/x_test.npy", x_test)

y_test = np.array(y_test)
np.save(image_test_path + "/y_test.npy", y_test)

# Build model 

In [ ]:

cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(101, activation='softmax'))

# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, batch_size=32, epochs=30, verbose=True)
print(cnn.summary())

In [ ]:
from tensorflow.keras import layers

lr = 3e-4
batch_size = 32
epochs = 200

model = tf.keras.Sequential(
    [
     layers.Conv2D(32,3,activation='relu',padding='same',name='conv1a', input_shape=[64,64,3]),
     layers.Conv2D(32,3,activation='relu',padding='same',name='conv1b'),
     layers.MaxPooling2D(2,2),
     layers.Conv2D(64,3,activation='relu',padding='same',name='conv2a'),
     layers.Conv2D(64,3,activation='relu',padding='same',name='conv2b'),
     layers.MaxPooling2D(2,2),
     layers.Conv2D(128,3,activation='relu',padding='same',name='conv3a'),
     layers.Conv2D(128,3,activation='relu',padding='same',name='conv3b'),
     layers.MaxPooling2D(2,2),
     layers.Flatten(),
     layers.Dense(1024,activation='relu',name='dense1'),
     layers.Dense(21,activation='softmax',name='z')
    ]
)
opt = tf.keras.optimizers.Adam(lr=lr)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics='accuracy')
history = model.fit(x=train_generator,validation_data=test_generator,batch_size=batch_size,epochs=epochs,verbose=True)



In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('iteration')
plt.ylabel('loss')
plt.title('Loss over time')
plt.legend(['train','val'])
plt.show()

# Evaluate model